In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import string
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold, cross_val_score, GridSearchCV 
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, plot_confusion_matrix, make_scorer, accuracy_score, f1_score, confusion_matrix, roc_auc_score, recall_score, precision_score, log_loss
pd.set_option('max_colwidth', 500)

In [ ]:
#loading  Dev data
data_dev=pd.read_csv('/content/drive/MyDrive/dev_with_labels.tsv',sep='\t')
data_dev.head()


,PID,Text_data,Label
0,dev_pid_1,"I enjoyed today, and I still am! Tomorrows depression can wait! : Today, I'm warm and cozy :) I even had some tea, to complement my sandwich!\nThis is a crazy time we live in, but it's only the beginning, with the world going to shambles, being alone feels werid.\nBut a stranger came through and saved me,\nOf course it's a debt I'll have to repay, but I'm grateful to have this debt!\nCrazy what a few dollars can do to change someones situation.\nBut these days, it's all about here and now!\n...",moderate
1,dev_pid_2,"I sorta tried to kill myself : I had a total breakdown in a fucking car park at 1am alone.i sat for balling my eye out for hours covered in blood and swallowed 32 pill. I'm in a little pain but I'm home now I've calmed down and I'm feeling a bit more relaxed. I've been so alone look lately, the only person who I can ""talk to"" is my ex who cheated, lied and often avoids me. I tried calling samaritans when I was outside however they waiting list was so long. I was planning on kept the pills an...",moderate
2,dev_pid_3,Best suicide method? : I like it quick and easy with no deformities haha!,moderate
3,dev_pid_4,"a story : I remember the time I'd get on my 3DS to play Nintendogs. I'd be at my grandparents house, having fun with my chihuahua. It was nice and calming, having no trouble when I need to get in a safe space. Peaceful music was played whenever I'd walk my dog. Competitive, yet smooth music played whenever I'd enter competitions. A fun experience was the least I could say. It was amazing.\nI had a dog once. Well, not exactly my dog, it was my grandparents. We called him Bugsy (Bugzy?) and he...",moderate
4,dev_pid_5,"The world only cares about beautiful people : I'm born ugly. I've always been ugly all my life and I don't think that I could ever become more beautiful. However, I used to think that if I tried my best and be a ""good"" person, people will like me too. But I'll never be treated like those who are beautiful. No matter how much I tried to become a excellent student, a helpful friend, others never really like me, no one cares about my existence. I have no real friends and I have never been in a ...",moderate


In [ ]:
data_dev['Label'].value_counts()

moderate          2306
not depression    1830
severe             360
Name: Label, dtype: int64

In [ ]:
#loading train data
data_train=pd.read_csv('/content/drive/MyDrive/train.tsv',sep='\t')
data_train.head()

,PID,Text_data,Label
0,train_pid_1,"Waiting for my mind to have a breakdown once the “New Year” feeling isn’t there anymore : I don’t know about anyone else, but I’m a little bit worried that I’ll go back to being depressed in a few days time or something. Last year, I tried not to have any breakdowns for the start of 2019. A mere 10 days later, I broke down crying. I wasn’t the same for that entire year. Up until December, where I was ok that month. Now I just wait... it’s a weird way to act and feel, but it feels a bit normal.",moderate
1,train_pid_2,"My new years resolution : I'm gonna get my ass into a therapists office, and if I dont become even a little bit happy, then I'm not dealing with this shit anymore.\n\nI'm not asking for a lot, just a little bit of serotonin is all I want",moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will be there last\nyear on earth because even wen your hammerd your Feeling like a moron thats depressed?,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my english is mostly self learned, so please excuse if I do anything wrong. So here it goes:\n\nI've been depressed for 4 years now and I'm only 16 and because I have no one in real life to talk to, I thought I'll just post it here. \n\nMy story starts with me being 12 years old and my best friend talking shit about me and telling I was a fake friend without me doing anything. For some reason he is still my best friend but he changed now luckily. Th...",moderate
4,train_pid_5,Sat in the dark and cried myself going into the new year. Great start to 2020 :,moderate


In [ ]:
#loading test data
data_test=pd.read_csv('/content/drive/MyDrive/test_data.tsv',sep='\t')
data_test.head()

,Pid,Text_data
0,test_pid_1,Im scared : This is it. I lie to myself every day and say ill make it out but i think this might just be it. \nI thought i wanted a girlfriend but i realize that what i really want is to feel loved. Ive spent 20 years on this earth but ive felt completely alone forever. My heart hurts. My subconscious taunts me every night only for me to wake up to the reality that im a loser who is not worth anyones time. \nIm scared i will live like this until the day i die.\nIm being torn from the inside ...
1,test_pid_2,"New to this but just wanted to vent : I just finally realized that I’m kind of in a bad mental state. There’s this feeling of dread in me. I don’t want to work so I made an excuse not to go to work. I kind of feel like all of this stem from being financially unstable where I’m living pay check to pay check, hating my job because of the toxic environment I just had enough of it where I’m feeling like I have to walk on eggshells around people and having people talk about me because I am not at..."
2,test_pid_3,I’m sad : It’s kinda always been an issue. I wouldn’t say it’s as bad as my peers but it’s definitely bad. There’s a lot that I obviously can’t write cause it would take a while but I’ll just say the grand scheme of things. So basically I’m not really happy. I haven’t been happy. It definitely started to get really bad last year but it was always an issue growing up. I was very emotional growing up and by the time I was in 4th or 5th grade I had self esteem issues. There wasn’t much of a rea...
3,test_pid_4,"Lonely but not alone. : All of my immediately family members is dead. My dad died 11 yrs ago and grandmother 9 yrs ago. My mom 7 months ago and grandfather a month ago. For six months, I was jobless and depressed. Now i have a job working as a service technician and i love it. But still i have episodes where i feel lonely and depressed and i miss my family but i have my fiancee and her daughter. My depression comes and goes like crazy. And makes it worse that i am still living in my mom old ..."
4,test_pid_5,"This year has been trash. : I dont know why I’m posting here. I don’t even know why I’m talking about it at all. In my family if you talk about your problems then you’re just attention seeking so I try not to. I guess I’m just tired of keeping it all to myself. This year I left my 4 year long abusive relationship. He raped me, he stalked me after we broke up, and harassed me relentlessly. I was scared to leave my own home. I was scared to even be *in* my own home. I was always worried that h..."


In [ ]:
data_train['Label'].value_counts()

moderate          6019
not depression    1971
severe             901
Name: Label, dtype: int64

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
english_stopwords = stopwords.words("english")
stopwords=english_stopwords

In [ ]:
def clean_text(text):
    #text = give_emoji_free_text(text)
    text = re.sub('@[a-zA-Z0-9]*', '', text)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) 
    text = "".join([word for word in text if not word.isdigit()])
    #tokens = re.split('\W+',text)
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

def feature_extraction(data):
    data['clean_text'] =data['Text_data'].apply(lambda x:clean_text(x))
    freq_rare = pd.Series(' '.join(data['clean_text']).split()).value_counts()[-20:]
    data['clean'] = data['clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_rare))

    data['char_count'] = data['Text_data'].apply(len)
    data['word_count'] = data['Text_data'].apply(lambda x: len(x.split()))
    data['word_density'] = data['char_count'] / (data['word_count']+1)
    data['punctuation_count'] = data['Text_data'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
    data['title_word_count'] = data['Text_data'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
    data['upper_case_word_count'] = data['Text_data'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

    data['noun_count'] = data['Text_data'].apply(lambda x: check_pos_tag(x, 'noun'))
    data['verb_count'] = data['Text_data'].apply(lambda x: check_pos_tag(x, 'verb'))
    data['adj_count'] = data['Text_data'].apply(lambda x: check_pos_tag(x, 'adj'))
    data['adv_count'] = data['Text_data'].apply(lambda x: check_pos_tag(x, 'adv'))
    data['pron_count'] = data['Text_data'].apply(lambda x: check_pos_tag(x, 'pron'))
    return data

In [ ]:
df_train = feature_extraction(data_train)
df_dev = feature_extraction(data_dev)
df_test = feature_extraction(data_test)

In [ ]:
df_test.shape

(3245, 15)

In [ ]:
# # characters level tf-idf
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3))#, max_features=5000)
tfidf_vect_ngram_chars.fit(data_train['clean'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(data_train['clean']) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(data_dev['clean']) 
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(data_test['clean'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:547: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  "The parameter 'token_pattern' will not be used"


In [ ]:
xtrain_tfidf_ngram_chars.shape,xtest_tfidf_ngram_chars.shape

((8891, 9024), (3245, 9024))

In [ ]:
# to construct tfidf 

count_vect = CountVectorizer()
cv_vect= count_vect.fit(df_train["clean"])
cv_vect_train= count_vect.transform(df_train["clean"])
cv_vect_test= count_vect.transform(df_dev["clean"])
cv_vect_test1= count_vect.transform(df_test["clean"])

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit(cv_vect_train)
X_tfidf_train = tfidf_transformer.transform(cv_vect_train)
X_tfidf_test = tfidf_transformer.transform(cv_vect_test)
X_tfidf_test1 = tfidf_transformer.transform(cv_vect_test1)

In [ ]:
X_tfidf_train.shape

(8891, 13169)

In [ ]:
#combining features
# train set
X_features_train = pd.concat([data_train['char_count'],data_train['word_count'],data_train['word_density'],data_train['adj_count'],data_train['adv_count'],data_train['pron_count'],data_train['noun_count'], pd.DataFrame(X_tfidf_train.toarray()),pd.DataFrame(xtrain_tfidf_ngram_chars.toarray())], axis=1)

#dev set
X_features_test = pd.concat([data_dev['char_count'],data_dev['word_count'],data_dev['word_density'],data_dev['adj_count'],data_dev['adv_count'],data_dev['pron_count'],data_dev['noun_count'],pd.DataFrame(X_tfidf_test.toarray()),pd.DataFrame(xvalid_tfidf_ngram_chars.toarray())], axis=1)

#test set
X_features_test1 = pd.concat([data_test['char_count'],data_test['word_count'],data_test['word_density'],data_test['adj_count'],data_test['adv_count'],data_test['pron_count'],data_test['noun_count'],pd.DataFrame(X_tfidf_test1.toarray()),pd.DataFrame(xtest_tfidf_ngram_chars.toarray())], axis=1)

In [ ]:
X_features_train

,char_count,word_count,word_density,adj_count,adv_count,pron_count,noun_count,0,1,2,...,9014,9015,9016,9017,9018,9019,9020,9021,9022,9023
0,498,101,4.882353,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,235,48,4.795918,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,146,27,5.214286,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3247,629,5.153968,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80,17,4.444444,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8886,615,101,6.029412,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8887,13339,2528,5.274417,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8888,1011,189,5.321053,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8889,1550,289,5.344828,0,0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
charcount=pd.concat([data_train['word_count']], axis=1)
charcount.shape

(8891, 1)

In [ ]:
X_features_train.shape,X_features_test1.shape

((8891, 22200), (3245, 22200))

Before resampling

In [ ]:
# Ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

estimator = [] 
estimator.append(('RF_classifier', RandomForestClassifier(n_estimators=100, max_depth=None, n_jobs=-1)))
estimator.append(('MNB_classifier',MultinomialNB()))
estimator.append(('MLP_classifier', MLPClassifier(random_state=1, max_iter=100)))
estimator.append(('GB_classifier', GradientBoostingClassifier(n_estimators=100, max_depth=None)))


# Voting Classifier
vot_soft = VotingClassifier(estimators = estimator, voting ='soft') 
vot_soft.fit(X_features_train,df_train["Label"]) 
y_pred_soft = vot_soft.predict(X_features_test) 

accuracy = accuracy_score(df_dev["Label"], y_pred_soft)
print("Test Accuracy:", round(accuracy*100, 4))
print("\n", classification_report(df_dev["Label"], y_pred_soft))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklea

Test Accuracy: 60.298

                 precision    recall  f1-score   support

      moderate       0.58      0.86      0.69      2306
not depression       0.69      0.36      0.48      1830
        severe       0.51      0.16      0.24       360

      accuracy                           0.60      4496
     macro avg       0.59      0.46      0.47      4496
  weighted avg       0.62      0.60      0.57      4496



In [ ]:
len(final_prediction)

3245

In [ ]:
# to create .csv file consisting of tweet ids and predicted labels
final_prediction = pd.DataFrame(data=final_prediction, columns=['final_prediction'])
Submisssion_en = pd.DataFrame()
Submisssion_en['pid'] = data_test['Pid']
Submisssion_en['class_label'] = final_prediction
Submisssion_en.to_csv('/content/drive/MyDrive/MUCS_submission2.tsv',index=False) 